In [1]:
import os, sys, ctypes
import win32com.client
import pandas as pd
from datetime import datetime
from slacker import Slacker
import time, calendar
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
import numpy as np
import re
import matplotlib.pylab as plt

In [2]:
cpStatus = win32com.client.Dispatch('CpUtil.CpCybos') # ! 설명 :연결확인
cpOhlc = win32com.client.Dispatch('CpSysDib.StockChart')

In [3]:
def get_ohlc(code, qty):
    """크레온 플러스 시스템 연결 상태를 점검한다."""
    # 관리자 권한으로 프로세스 실행 여부
    if not ctypes.windll.shell32.IsUserAnAdmin():
        print('error')
        return False
 
    # 연결 여부 체크
    if (cpStatus.IsConnect == 0):
        print('error')
        return False
    """인자로 받은 종목의 OHLC 가격 정보를 qty 개수만큼 반환한다."""
    cpOhlc.SetInputValue(0, code)           # 종목코드
    cpOhlc.SetInputValue(1, ord('2'))        # ! (요청구분(1), 1:기간 or 2:개수)
    cpOhlc.SetInputValue(4, qty)             # ! 요청개수(몇일치 데이터를 가져올지 정함)
    cpOhlc.SetInputValue(5, [0, 2, 3, 4, 5]) # 0:날짜, 2~5:OHLC
    cpOhlc.SetInputValue(6, ord('D'))        # ! (차트구분(6),D:일단위)
    cpOhlc.SetInputValue(9, ord('1'))        # 0:무수정주가, 1:수정주가(#-유상ㆍ무상 증자나 액면 분할 고려 주식 가격의 차이를 수정한 주가.)
    cpOhlc.BlockRequest()
    count = cpOhlc.GetHeaderValue(3)   # 3:수신개수
    columns = ['open', 'high', 'low', 'close']
    index = []
    rows = []
    '''요청한 데이터는 (5,[0,2,3,4,5])같이 ohlc정보만이다. 나머지는 정보 세팅이다.'''
    for i in range(count): 
        index.append(cpOhlc.GetDataValue(0, i)) # ! (type,index) type에 해당하는 데이터 반환(0번째로 요청한데이터:날짜),요청한 데이터의 index(여기선 날짜)
        rows.append([cpOhlc.GetDataValue(1, i), cpOhlc.GetDataValue(2, i),
            cpOhlc.GetDataValue(3, i), cpOhlc.GetDataValue(4, i)]) 
    df = pd.DataFrame(rows, columns=columns, index=index) 
    return df

def get_target_price(code,day):
    """과거의 매수 목표가를 반환한다."""
    ohlc = get_ohlc(code, day+1) #day+1만큼의 데이터 day만큼 연산값얻기위해 +1해줌
    target_price=[]
    for i in range(day):              
        #마지막 i의 i+1은 ohlc데이터에 없기 때문에 1개를 줄여야한다. IndexError: single positional indexer is out-of-bounds 뜸          
        lastday = ohlc.iloc[i+1]                                     
        today_open = lastday[3]         # ! close
            
        lastday_high = lastday[1]
        lastday_low = lastday[2]
        target_price.append(today_open + (lastday_high - lastday_low) * 0.1) #전일 시가와 종가 사이 일정범위내의 주가상승분일때 그때 매수 여기는 0.1배일때
    df=get_ohlc(code,day)
    return target_price,df

if __name__=='__main__':
    day=60
    code='A091170'
    target_price,df=get_target_price(code,day)

In [4]:
df['target_price']=np.ceil(np.array(target_price)).astype(int) # ! target_price를 올림해준다.
df_S=df[(df['target_price']<=df['high'])&(df['target_price']>=df['low'])] # ! target_price에 살수있을때만 남겨놈
target_prob=len(df_S)/len(df)

In [5]:
# ! 5원 단위 거래가능이므로 1~4라면 5, 6~9이라면 10으로 맞춰준다.
df_S['target_price']=df_S['target_price'].astype(int).map(lambda x: np.ceil(x/5)*5).astype(int)


C:\python38-32\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [6]:
# ! 1%먹고 팔기 성공, 실패하면 종가판매
df_S['sell_price']=np.where((df_S['target_price']*1.01>=df_S['low'])&(df_S['target_price']*1.01<=df_S['high']),df_S['target_price']*1.01,df_S['close'])
onesell_prob=len(df_S[df_S['sell_price']==df_S['target_price']])/len(df) # ! 1%먹은 확률

In [7]:
df_S['sell_price']=np.ceil(np.ceil(df_S['sell_price'])/5)*5

In [8]:
df_S.astype(int)

,open,high,low,close,target_price,sell_price
20210813,7550,7550,7375,7435,7530,7435
20210812,7525,7550,7480,7520,7485,7520
20210810,7445,7455,7350,7395,7440,7395
20210806,7390,7435,7315,7330,7350,7425
20210805,7390,7420,7340,7340,7420,7340
20210803,7245,7280,7195,7280,7270,7280
20210802,7320,7335,7200,7255,7325,7255
20210728,7300,7425,7290,7410,7345,7420
20210726,7385,7385,7265,7280,7370,7280
20210723,7375,7380,7280,7360,7375,7360


In [11]:
def returns(price):
    qty=price/(df_S['target_price']*1.00019)
    day_returns=qty*((df_S['target_price']*1.00019)-(df_S['sell_price']*0.99981))
    returns=day_returns.sum()
    return returns,day_returns

returns,day_returns=returns(1000000)
print(returns)

24166.846465645092
